In [1]:
import pyspark
import dxpy
import dxdata
import toml

from distutils.version import LooseVersion
from subprocess import run
from pathlib import Path

import hail as hl

In [2]:
sc = pyspark.SparkContext()
spark = pyspark.sql.SparkSession(sc)

In [41]:
with open("../config.toml") as f:
    conf = toml.load(f)

RAW_REL_FILE = conf["SAMPLE_QC"]["UKB_REL_DAT_FILE"]
REL_FILE = conf["SAMPLE_QC"]["REL_FILE"]
SAMP_FILE = conf["SAMPLE_QC"]["SAMP_FILE"]

MAX_KINSHIP = conf["SAMPLE_QC"]["MAX_KINSHIP"]

LOG_FILE = Path(conf["IMPORT"]["LOG_DIR"], f"sample_filters.log").resolve().__str__()
TMP_DIR = Path(conf["EXPORT"]["TMP_DIR"])
DATA_DIR = conf["SAMPLE_QC"]["DATA_DIR"]

In [5]:
hl.init(sc=sc, default_reference='GRCh38', log=LOG_FILE)

pip-installed Hail requires additional configuration options in Spark referring
  to the path to the Hail Python module directory HAIL_DIR,
  e.g. /path/to/python/site-packages/hail:
    spark.jars=HAIL_DIR/hail-all-spark.jar
    spark.driver.extraClassPath=HAIL_DIR/hail-all-spark.jar
    spark.executor.extraClassPath=./hail-all-spark.jarRunning on Apache Spark version 2.4.4
SparkUI available at http://ip-10-60-170-42.eu-west-2.compute.internal:8081
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.61-3c86d3ba497a
LOGGING: writing to /opt/notebooks/gogoGPCR/hail_logs/sample_filters.log


In [6]:
dispensed_database_name = dxpy.find_one_data_object(classname="database", name="app*", folder="/", name_mode="glob", describe=True)["describe"]["name"]
dispensed_dataset_id = dxpy.find_one_data_object(typename="Dataset", name="app*.dataset", folder="/", name_mode="glob")["id"]

In [7]:
dataset = dxdata.load_dataset(id=dispensed_dataset_id)

In [8]:
participant = dataset["participant"]

In [8]:
def fields_for_id(field_id):

    field_id = str(field_id)
    fields = participant.find_fields(name_regex=r'^p{}(_i\d+)?(_a\d+)?$'.format(field_id))
    
    return sorted(fields, key=lambda f: LooseVersion(f.name))

In [9]:
fields = ["22027", "22019", "22006", "22021"]
field_names = [fields_for_id(id) for id in fields] #fields_for_id("22027") + fields_for_id("22019") + fields_for_id("22006") + fields_for_id("22021")
field_names = ["eid"] + [field.name for fields in field_names for field in fields]

In [10]:
df = participant.retrieve_fields(names=field_names, engine=dxdata.connect(), coding_values="replace")

In [11]:
df.show(5, truncate=False)

+-------+------+------+---------+--------------------------------+
|eid    |p22027|p22019|p22006   |p22021                          |
+-------+------+------+---------+--------------------------------+
|3888244|null  |null  |Caucasian|No kinship found                |
|1795659|null  |null  |Caucasian|No kinship found                |
|2084720|null  |null  |Caucasian|At least one relative identified|
|3742232|null  |null  |Caucasian|At least one relative identified|
|1094442|null  |null  |Caucasian|At least one relative identified|
+-------+------+------+---------+--------------------------------+
only showing top 5 rows



In [12]:
df = df.filter((df.p22006 == "Caucasian") &
          (df.p22027.isNull()) &
          (df.p22019.isNull()) &
          (df.p22021 != "Participant excluded from kinship inference process") & 
          (df.p22021 != "Ten or more third-degree relatives identified"))

df.count()

In [15]:
SAMP_PATH = TMP_DIR / SAMP_FILE
df = df.select("eid")
df.toPandas().to_csv(SAMP_PATH, index=False, sep="\t")

In [16]:
run(["dx", "upload", SAMP_PATH, "--path", DATA_DIR], check = True, shell = False)

CompletedProcess(args=['dx', 'upload', '/opt/notebooks/gogoGPCR/data/samples_to_keep.tsv', '--path', '/data/processed/'], returncode=0)

In [25]:
rel = hl.import_table(
        "file:" + RAW_REL_FILE,
        delimiter=" ",
        impute=True,
        types={"ID1": "str", "ID2": "str"},
    )

rel = rel.filter(rel.Kinship > MAX_KINSHIP, keep=True)

related_samples_to_remove = hl.maximal_independent_set(
    i=rel.ID1,
    j=rel.ID2,
    keep=False,
).rename({"node": "eid"}).key_by("eid")

related_samples_to_remove.count()

2021-09-28 08:37:16 Hail: INFO: Reading table to impute column types
2021-09-28 08:37:20 Hail: INFO: Finished type imputation
  Loading field 'ID1' as type str (user-supplied type)
  Loading field 'ID2' as type str (user-supplied type)
  Loading field 'HetHet' as type float64 (imputed)
  Loading field 'IBS0' as type float64 (imputed)
  Loading field 'Kinship' as type float64 (imputed)
2021-09-28 08:37:23 Hail: INFO: wrote table with 40241 rows in 1 partition to /tmp/0c7UNWoPsEiWd4rmvYOf9b
    Total size: 622.02 KiB
    * Rows: 622.01 KiB
    * Globals: 11.00 B
    * Smallest partition: 40241 rows (622.01 KiB)
    * Largest partition:  40241 rows (622.01 KiB)
2021-09-28 08:37:25 Hail: INFO: Ordering unsorted dataset with network shuffle


34824

In [47]:
REL_PATH = (TMP_DIR / REL_FILE).resolve().__str__()
related_samples_to_remove.export("file:" + REL_PATH)

2021-09-28 08:43:56 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-09-28 08:43:56 Hail: INFO: merging 1 files totalling 272.1K...
2021-09-28 08:43:56 Hail: INFO: while writing:
    file:/opt/notebooks/gogoGPCR/tmp/samples_to_remove.tsv
  merge time: 43.536ms


In [48]:
run(["dx", "upload", REL_PATH, "--path", DATA_DIR, check = True, shell = False)

CompletedProcess(args=['dx', 'upload', '/opt/notebooks/gogoGPCR/tmp/samples_to_remove.tsv', '--path', '/data/processed/'], returncode=0)